In [ ]:
class Scheduler:
    def __init__(self):
        self.tasks = {}
        self.dependencies = {}
        self.done = set()

    def add_task(self, task, dependencies=None):
        self.tasks[task] = False
        self.dependencies[task] = set(dependencies) if dependencies else set()

    def add_and_task(self, task, dependencies=None):
        self.add_task(task, dependencies)

    def add_or_task(self, task, dependencies=None):
        self.add_task(task, dependencies)
        self.dependencies[task].add(None)  # Add a dummy dependency

    @property
    def available_tasks(self):
        return [task for task in self.tasks if self.dependencies[task].issubset(self.done)]

    @property
    def uncompleted(self):
        return [task for task in self.tasks if not self.tasks[task]]

    def mark_completed(self, task):
        self.tasks[task] = True
        self.done.add(task)
        for dependent_task in self.dependencies:
            if self.dependencies[dependent_task] and task in self.dependencies[dependent_task]:
                self.dependencies[dependent_task].remove(task)

    def show(self):
        g = nx.DiGraph()
        g.add_nodes_from(self.tasks.keys())
        for task, dependencies in self.dependencies.items():
            for dependency in dependencies:
                g.add_edge(dependency, task)
        nx.draw(g, with_labels=True)

scheduler = Scheduler()

scheduler.add_and_task('dice onions')
scheduler.add_and_task('dice pancetta')
scheduler.add_task('warm water')
scheduler.add_task('beat eggs', dependencies=['dice onions', 'dice pancetta'])
scheduler.add_task('cook pancetta and onions', dependencies=['dice onions', 'dice pancetta'])
scheduler.add_task('boil water', dependencies=['warm water'])
scheduler.add_task('cook pasta', dependencies=['boil water'])
scheduler.add_task('colander pasta', dependencies=['cook pasta'])
scheduler.add_or_task('prepare carbonara', dependencies=['cook pancetta and onions', 'beat eggs'])
scheduler.add_task('mix carbonara', dependencies=['prepare carbonara', 'colander pasta'])

scheduler.mark_completed('dice onions')
scheduler.mark_completed('dice pancetta')
scheduler.mark_completed('warm water')
scheduler.mark_completed('cook pancetta and onions')
scheduler.mark_completed('boil water')
scheduler.mark_completed('cook pasta')

print(scheduler.available_tasks)  # ['beat eggs', 'colander pasta', 'prepare carbonara']
print(scheduler.uncompleted)  # ['mix carbonara']

scheduler.mark_completed('beat eggs')
scheduler.mark_completed('colander pasta')
scheduler.mark_completed('prepare carbonara')

print(scheduler.available_tasks)  # ['mix carbonara']
print(scheduler.uncompleted)  # []
